# Automated ML

In [1]:
from azureml.core import Experiment, Workspace

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone'

experiment=Experiment(ws, experiment_name)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Zindi: DSN AI Bootcamp Qualification Hackathon [data](https://zindi.africa/hackathons/dsn-ai-bootcamp-qualification-hackathon/data)

In [3]:
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

from scripts.cleaning import clean_data

In [4]:
loan_dataset = Dataset.get_by_name(ws, name='loan_dataset')

In [5]:
loan_dataset.to_pandas_dataframe().head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [6]:
clean_loan_dataset = clean_data(loan_dataset)

# Stratified train_test_split because dataset is imbalanced
train, test = train_test_split(clean_loan_dataset, test_size=0.3, stratify=clean_loan_dataset.default_status, random_state=42)
train.head()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
4582,3500.0,0.43040,0.0000,0.0,0.0000,49014.0,3915674.0,849965.0,6083182.0,9494323.0,...,0.200002,16.16,0.614328,0.0,0.0,0,334.375561,1.000000,0.126240,1
50148,3550.0,0.10010,1.2900,0.0,0.0000,269966.0,9008851.0,9008851.0,38148452.0,22780618.0,...,0.046805,19.19,0.616856,0.0,0.0,1,4066.551346,0.000000,0.989586,0
30190,3426.0,0.81005,0.8574,0.0,0.0000,0.0,37102431.0,327538.0,4947302.0,38894943.0,...,0.378125,12.12,0.693056,0.0,0.0,0,316.797746,2.691935,0.102668,0
54728,3502.0,0.71285,0.0000,0.0,0.0000,8558.0,5231661.0,146653.0,12316518.0,10782691.0,...,0.066671,20.20,0.406624,0.0,0.0,0,650.811063,7.000000,0.255596,0
38352,3400.0,0.41410,2.3760,0.0,2.1528,0.0,6003437.0,228343.0,4444714.0,6815280.0,...,0.366663,9.09,0.583440,0.0,0.0,0,158.716495,1.822512,0.114156,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
import logging

from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

In [8]:
automl_settings = {
    "featurization": "auto",
    "n_cross_validations": 4,
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO
}

automl_config = AutoMLConfig(
    task="classification",
    training_data=train,
    label_column_name="default_status",
    primary_metric="AUC_weighted",
    **automl_settings
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_718264c0-8f6b-4bc7-bad4-a00e4ed47f2a

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

In [11]:
from azureml.core.model import Model
import joblib

from utils import print_model

In [12]:
automl_run, best_automl_model = remote_run.get_output()

In [13]:
print(automl_run)

Run(Experiment: capstone,
Id: AutoML_718264c0-8f6b-4bc7-bad4-a00e4ed47f2a_25,
Type: None,
Status: Completed)


In [14]:
print_model(best_automl_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['23', '11', '17', '3', '4', '19', '12', '14', '1', '21'],
 'weights': [0.13333333333333333,
             0.06666666666666667,
             0.13333333333333333,
             0.2,
             0.06666666666666667,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667]}

23 - maxabsscaler
{'copy': True}

23 - sgdclassifierwrapper
{'alpha': 3.8776122448979593,
 'class_weight': 'balanced',
 'eta0': 0.01,
 'fit_intercept': True,
 'l1_ratio': 0.44897959183673464,
 'learning_rate': 'invscaling',
 'loss': 'log',
 

In [15]:
#TODO: Save the best model
joblib.dump(best_automl_model, "outputs/automl_model.joblib")

['outputs/automl_model.joblib']

In [17]:
model = Model.register(
    workspace=ws,
    model_path="outputs/automl_model.joblib",
    model_name="AutoML_Voting_Ensemble",
    tags={"accuracy": 0.8543},
    description="default_status prediction model"
)

Registering model AutoML_Voting_Ensemble


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service